In [11]:
from pyspark.sql import SparkSession  # Spark SQL 작업을 위한 SparkSession 임포트
from pyspark.sql.functions import  # Spark SQL 함수들 임포트 explode, col, when

spark = (SparkSession.builder  # SparkSession 빌더 패턴 시작
         .appName("handle-nulls")  # 애플리케이션 이름 설정
         .master("spark://spark-master:7077")  # Spark 마스터 URL 설정
         .config("spark.executor.memory", "512m")  # Spark 설정 옵션
         .getOrCreate()  # SparkSession 생성 또는 기존 세션 반환)

spark.sparkContext.setLogLevel("ERROR")  # 로그 레벨을 ERROR로 설정

In [12]:
df = (spark.read.format("json")  # JSON 형식으로 데이터 읽기
      .option("multiLine", "true")  # 여러 줄 JSON 처리
      .load(  # 파일 로드"../data/nobel_prizes.json"))

df_flattened = (
    df
    .withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"laureates",explode(col("laureates")))
    .select(  # 컬럼 선택col("category")
            ,col(  # 컬럼 참조"year")
            ,col(  # 컬럼 참조"overallMotivation")
            ,col(  # 컬럼 참조"laureates.id")
            ,col(  # 컬럼 참조"laureates.firstname")
            ,col(  # 컬럼 참조"laureates.surname")
            ,col(  # 컬럼 참조"laureates.share")
            ,col(  # 컬럼 참조"laureates.motivation")))

In [13]:
# Dropping rows with null values
df_dropna = df_flattened.dropna(  # null 값이 있는 행 제거)

# 출력ing the DataFrame after dropping null values
df_dropna.show()  # DataFrame 내용 출력

+---------+----+--------------------+----+----------+----------+-----+--------------------+
| category|year|   overallMotivation|  id| firstname|   surname|share|          motivation|
+---------+----+--------------------+----+----------+----------+-----+--------------------+
|  physics|2021|"for groundbreaki...| 999|   Syukuro|    Manabe|    4|"for the physical...|
|  physics|2021|"for groundbreaki...|1000|     Klaus|Hasselmann|    4|"for the physical...|
|  physics|2021|"for groundbreaki...|1001|   Giorgio|    Parisi|    2|"for the discover...|
|  physics|2019|"for contribution...| 973|     James|   Peebles|    2|"for theoretical ...|
|  physics|2019|"for contribution...| 974|    Michel|     Mayor|    4|"for the discover...|
|  physics|2019|"for contribution...| 975|    Didier|    Queloz|    4|"for the discover...|
|  physics|2018|"for groundbreaki...| 960|    Arthur|    Ashkin|    2|"for the optical ...|
|  physics|2018|"for groundbreaki...| 961|    Gérard|    Mourou|    4|"for their

In [14]:
# Filling null values with a specific value
df_fillna = df_flattened.fillna(  # null 값 채우기"N/A")

# 출력ing the DataFrame after filling null values
df_fillna.show()  # DataFrame 내용 출력


+----------+----+-----------------+----+--------------------+-----------+-----+--------------------+
|  category|year|overallMotivation|  id|           firstname|    surname|share|          motivation|
+----------+----+-----------------+----+--------------------+-----------+-----+--------------------+
| chemistry|2022|              N/A|1015|             Carolyn|   Bertozzi|    3|"for the developm...|
| chemistry|2022|              N/A|1016|              Morten|     Meldal|    3|"for the developm...|
| chemistry|2022|              N/A| 743|               Barry|  Sharpless|    3|"for the developm...|
| economics|2022|              N/A|1021|                 Ben|   Bernanke|    3|"for research on ...|
| economics|2022|              N/A|1022|             Douglas|    Diamond|    3|"for research on ...|
| economics|2022|              N/A|1023|              Philip|     Dybvig|    3|"for research on ...|
|literature|2022|              N/A|1017|               Annie|     Ernaux|    1|"for the cou

In [15]:
# Replacing null values based on conditions
df_replace = (
    df_flattened.withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"category", when(col("category").isNull(), "").otherwise(col("category")))
    .withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"overallMotivation", when(col("overallMotivation").isNull(), "").otherwise(col("overallMotivation")))
    .withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"firstname", when(col("firstname").isNull(), "").otherwise(col("firstname")))
    .withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"surname", when(col("surname").isNull(), "").otherwise(col("surname")))
    .withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"year", when(col("year").isNull(), 9999).otherwise(col("year"))))

# 출력ing the DataFrame after replacing null values
df_replace.show()  # DataFrame 내용 출력


+----------+----+-----------------+----+--------------------+-----------+-----+--------------------+
|  category|year|overallMotivation|  id|           firstname|    surname|share|          motivation|
+----------+----+-----------------+----+--------------------+-----------+-----+--------------------+
| chemistry|2022|                 |1015|             Carolyn|   Bertozzi|    3|"for the developm...|
| chemistry|2022|                 |1016|              Morten|     Meldal|    3|"for the developm...|
| chemistry|2022|                 | 743|               Barry|  Sharpless|    3|"for the developm...|
| economics|2022|                 |1021|                 Ben|   Bernanke|    3|"for research on ...|
| economics|2022|                 |1022|             Douglas|    Diamond|    3|"for research on ...|
| economics|2022|                 |1023|              Philip|     Dybvig|    3|"for research on ...|
|literature|2022|                 |1017|               Annie|     Ernaux|    1|"for the cou

### Handling null values in user-defined functions (UDFs)

In [16]:
from pyspark.sql import SparkSession  # Spark SQL 작업을 위한 SparkSession 임포트
from pyspark.sql.functions import  # Spark SQL 함수들 임포트 udf
from pyspark.sql.types import  # Spark SQL 데이터 타입 임포트 StringType

# Sample DataFrame with null values
data = [("John", 25), ("Alice", None), ("Bob", 30)]
df = spark.createDataFrame(data, ["name", "age"])

# Define a UDF to handle null values
def process_name(name):
    if name is None:
        return "Unknown"
    else:
        return name.upper(  # 대문자로 변환)

# Register the UDF
process_name_udf = udf(  # 사용자 정의 함수 생성process_name, StringType())

# 적용 the UDF to the DataFrame
df_with_processed_names = df.withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"processed_name", process_name_udf(df["name"]))

# 표시 the resulting DataFrame
df_with_processed_names.show()  # DataFrame 내용 출력


+-----+----+--------------+
| name| age|processed_name|
+-----+----+--------------+
| John|  25|          JOHN|
|Alice|null|         ALICE|
|  Bob|  30|           BOB|
+-----+----+--------------+



### Handling null values in machine learning pipelines

In [17]:
from pyspark.sql import SparkSession  # Spark SQL 작업을 위한 SparkSession 임포트
from pyspark.ml.feature import Imputer

# 생성 a sample DataFrame with missing values
data = [
    (1, 2.0),
    (2, None),
    (3, 5.0),
    (4, None),
    (5, 7.0)
]
df = spark.createDataFrame(data, ["id", "value"])

# 생성 an instance of Imputer and specify the input/output columns
imputer = Imputer(inputCols=["value"], outputCols=["imputed_value"])

# Fit the imputer to the data and transform the DataFrame
imputer_model = imputer.fit(df)
imputed_df = imputer_model.transform(  # 배열 변환df)

# 표시 the resulting DataFrame
imputed_df.show()  # DataFrame 내용 출력

+---+-----+-----------------+
| id|value|    imputed_value|
+---+-----+-----------------+
|  1|  2.0|              2.0|
|  2| null|4.666666666666667|
|  3|  5.0|              5.0|
|  4| null|4.666666666666667|
|  5|  7.0|              7.0|
+---+-----+-----------------+



In [18]:
spark.stop()  # Spark 세션 종료 - 리소스 정리